In [1]:
import dash
import pandas as pd
import numpy as np
import plotly.express as px
from dash import dcc, html, no_update
from dash.dependencies import Input, Output

In [2]:
text = "Sed ut perspiciatis, unde omnis iste natus error sit voluptatem accusantium doloremque laudantium,totam rem aperiam eaque ipsa, quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt, explicabo. Nemo enim ipsam voluptatem, quia voluptas sit, aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos, qui ratione voluptatem sequi nesciunt, neque porro quisquam est, qui dolorem ipsum, quia dolor sit, amet, consectetur, adipisci velit, sed quia non numquam eius modi tempora incidunt, ut labore et dolore magnam aliquam quaerat voluptatem. Ut enim ad minima veniam, quis nostrum exercitationem ullam corporis suscipit laboriosam, nisi ut aliquid ex ea commodi consequatur? Quis autem vel eum iure reprehenderit, qui in ea voluptate velit esse, quam nihil molestiae consequatur, vel illum, qui dolorem eum fugiat, quo voluptas nulla pariatur? At vero eos et accusamus et iusto odio dignissimos ducimus, qui blanditiis praesentium voluptatum deleniti atque corrupti, quos dolores et quas molestias excepturi sint, obcaecati cupiditate non provident, similique sunt in culpa, qui officia deserunt mollitia animi, id est laborum et dolorum fuga. Et harum quidem rerum facilis est et expedita distinctio. Nam libero tempore, cum soluta nobis est eligendi optio, cumque nihil impedit, quo minus id, quod maxime placeat, facere possimus, omnis voluptas assumenda est, omnis dolor repellendus. Temporibus autem quibusdam et aut officiis debitis aut rerum necessitatibus saepe eveniet, ut et voluptates repudiandae sint et molestiae non recusandae. Itaque earum rerum hic tenetur a sapiente delectus, ut aut reiciendis voluptatibus maiores alias consequatur aut perferendis doloribus asperiores repellat."
text = text.split(",")
numbers = pd.DataFrame({"sample": np.random.randn(len(text)), "description": text})

In [3]:
numbers["word_count"] = numbers["description"].apply(lambda x: len(x.split()))

In [4]:
numbers.head(5)

,sample,description,word_count
0,-0.355155,Sed ut perspiciatis,3
1,1.168412,unde omnis iste natus error sit voluptatem ac...,10
2,-1.723820,totam rem aperiam eaque ipsa,5
3,-0.601270,quae ab illo inventore veritatis et quasi arc...,12
4,1.055939,explicabo. Nemo enim ipsam voluptatem,5


In [5]:
chart = px.scatter(
        numbers,
        x=numbers.index,
        y="sample",
        template='none',
        size=[10 for item in numbers["sample"]],
        title="Hover over to see tooltip in action"
    )

chart.update_traces(hoverinfo="none", hovertemplate=None)
chart.show()

In [6]:
# Initialize Dash app
app = dash.Dash()

# App layout
app.layout = html.Div(children=[
        dcc.Graph(id="graph", figure=chart),
        dcc.Tooltip(id="tooltip"),
    ],
    style={
        "fontFamily": "Verdana",
        "padding": "20px",
        "background-color": "#f7f7f7",
        "color": "#444",
    },
)

# Tooltip callback
@app.callback(
    [Output("tooltip", "show"), Output("tooltip", "bbox"), Output("tooltip", "children")],
    [Input("graph", "hoverData")],
)
def display_tooltip(data):
    """
    Update tooltip with the word count of the hovered text fragment.

    Args:
        data: Hover data containing information about the hovered point.

    Returns:
        Tooltip visibility, bounding box, and content.
    """
    if data is None:
        return False, no_update, no_update

    # Get hovered point information
    hovered_point = data["points"][0]
    bbox = hovered_point["bbox"]

    # Tooltip content
    children = [
        html.H4("Word Count Info", style={"margin-bottom": "10px", "color": "#333"}),
        html.P(f"Number of words: {numbers.iloc[hovered_point["x"]]["word_count"]}", style={"font-size": "16px"}),
        html.P(numbers.iloc[hovered_point["x"]]["description"], style={"font-size": "16px"}),
    ]

    return True, bbox, children

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)